# Install Libs

In [1]:
!pip install --upgrade boto3
!pip install wget
!brew install ffmpeg
import IPython.display
IPython.display.clear_output()

# Import Libs

In [3]:
import boto3
import dill
import sys, os, shutil
import subprocess

# Define Paths

In [76]:
path_home_dir = os.path.expanduser(os.path.join('~','Desktop'))
path_report_dir = os.path.join(path_home_dir, 'AWS_Transcribe')

if os.path.isdir(path_report_dir)==False:
    os.makedirs(path_report_dir)

# Load Access Keys

In [5]:
path_access_keys_file = '../accessKeys.csv'

access_keys = pd.read_csv(path_access_keys_file)

personal_access_key = access_keys['Access key ID'].iloc[0]
secret_access_key = access_keys["Secret access key"].iloc[0]

# Upload to Bucket

In [15]:
region = 'us-west-1'
s3_client = boto3.client(service_name = 's3',
                      region_name = region,
                      aws_access_key_id = personal_access_key,
                      aws_secret_access_key = secret_access_key)

In [16]:
username = 'jl'
bucket = username + '-' + os.path.split(path_report_dir)[1].lower().replace('_','-')
bucket

'jl-aws-transcribe'

In [19]:
path_audio = './speech_audio/OSR_us_000_0010_8k.wav'
audio_filename = 'OSR_us_000_0010_8k.wav'

In [20]:
# Create Bucket if not alread present
if bucket not in [bucket['Name'] for bucket in s3_client.list_buckets()['Buckets']]:
    s3_client.create_bucket(Bucket = bucket, 
                         CreateBucketConfiguration =  {'LocationConstraint': region} )

#upload if the video isn't already in the bucket
if 'Contents' in s3_client.list_objects(Bucket = bucket).keys():
    if audio_filename not in [obj['Key'] for obj in s3_client.list_objects(Bucket=bucket)['Contents']]:
        s3_client.upload_file(path_audio, bucket, audio_filename)
else:
    s3_client.upload_file(path_audio, bucket, audio_filename)

# Start Transcribe Client

In [21]:

trans_client =  boto3.client(service_name = 'transcribe',
                                      region_name = region, 
                                      aws_access_key_id = personal_access_key,
                                      aws_secret_access_key = secret_access_key)

In [82]:

def run_transcription_job(audio_filename,
                          TranscriptionJobName, 
                          s3_client, trans_client,
                          region,
                          bucket, path_output_file):
    """
    Run a transcription job, download the resulting json, and delete the job after completion
    """
    
    response = trans_client.start_transcription_job(TranscriptionJobName=TranscriptionJobName,
                                         LanguageCode='en-US',
                                         Media={'MediaFileUri':'https://s3-'+region+'.amazonaws.com/'+bucket+'/'+audio_filename},
                                         OutputBucketName=bucket
                                                   )
    progress = '.'
    while response['TranscriptionJob']['TranscriptionJobStatus'] == 'IN_PROGRESS':
        response = trans_client.get_transcription_job(TranscriptionJobName=TranscriptionJobName)
        progress = progress + '.'
        print(progress,end='\r')
    print(progress,response['TranscriptionJob']['TranscriptionJobStatus'],'!')
    
    s3_client.download_file(Bucket = bucket,
                         Key = TranscriptionJobName+'.json',
                         Filename = path_output_file)

    trans_client.delete_transcription_job(TranscriptionJobName='string')
    
    print('transcription job json ouput saved to',path_output_file)

    
    
TranscriptionJobName='string'
path_output_file = os.path.join(path_report_dir,TranscriptionJobName+'.json')
   
run_transcription_job(audio_filename,
                          TranscriptionJobName, 
                          s3_client, trans_client,
                          region,
                          bucket, path_output_file)

.......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................... COMPLETED !
transcription job json ouput saved to /Users/johnleonard/Desktop/AWS_Transcribe/string.json


In [91]:
file = open(path_output_file, 'r')
Transcription = json.load(file)
file.close

Transcription['results']['transcripts'][0]['transcript']

"the birds. Canoes lit on this move. Planks glue the sheet to the doctor blue background. It is easy to tell the death over well, these days, the chicken make is a very dish. Rice's often served in Randall's The juice of limits makes fine punch. The boxes don't decide the park took the Hobbs or such popcorn and garbage. Four hours of study work faced us. A. Large size and stockings is hard to sell."